# 海流作用力与翻转力矩

In [1]:
import numpy as np
from scipy.integrate import quad

# 常量定义
rho = 1025  # 海水的质量密度（kg/m^3）
Cd = 1.05  # 阻力系数
A_y = 0.06  # 平行于y轴的结构件单位长度的投影面积（m^2/m）
A_z = 0.06  # 平行于z轴的结构件单位长度的投影面积（m^2/m）
L_y_values = [2.06, 2.9, 2.9]  # 平行于y轴的结构件水平方向的长度（m）
z_positions = [1.0, 1.7, 2.2]  # 平行于y轴的结构件距底的位置（m）
F_Dy_total = 0
M_tilt_y_total=0

### 修改海流流速和平均水深

In [2]:
u_cs = 2.0  # 海面的海流速度（m/s）
h = 50  # 平均水深（m）

In [3]:
for i, z in enumerate(z_positions):
    ucz = u_cs * (z / h)**(1/7)  # 计算流速
    L_y = L_y_values[i]  # 取得对应位置的结构件水平方向的长度
    F_Dy = 0.5 * rho * Cd * A_y * ucz**2 * L_y  # 计算受力
    # 考虑前后两根结构件
    F_Dy_total += 2 * F_Dy
    M_tilt_y = F_Dy * z  # 计算翻转力矩
    M_tilt_y_total += 2 * M_tilt_y
# 计算平行于z轴的结构件受力的积分函数
def integrand(z):
    ucz = u_cs * (z / h)**(1/7)
    return 0.5 * rho * Cd * A_z * ucz**2
F_Dz_total, _ = quad(integrand, 0, 2.2)
M_tilt_z, _ = quad(lambda z: F_Dz_total, 0, 2.2)
# 乘以支腿数量
num_legs = 4  # 假设有4根支腿
F_Dz_total *= num_legs
M_tilt_z_total = num_legs*M_tilt_z
F_D_total = F_Dy_total + F_Dz_total
M_tilt_total = M_tilt_y_total + M_tilt_z_total
print("平行于y轴的结构件受力（N）:", F_Dy_total)
print("平行于z轴的结构件受力（N）:", F_Dz_total)
print("平台总受力（N）:", F_D_total)
print("平行于y轴的结构件受力产生的翻转力矩（Nm）:", M_tilt_y_total)
print("平行于z轴的结构件受力产生的翻转力矩（Nm）:", M_tilt_z_total)
print("平台总的翻转力矩（Nm）:", M_tilt_total)

平行于y轴的结构件受力（N）: 765.9326601502564
平行于z轴的结构件受力（N）: 362.1159576685858
平台总受力（N）: 1128.048617818842
平行于y轴的结构件受力产生的翻转力矩（Nm）: 1333.7082478940076
平行于z轴的结构件受力产生的翻转力矩（Nm）: 796.6551068708889
平台总的翻转力矩（Nm）: 2130.3633547648965


# 平台配载

## 序号1-4全填0(有时候会错，但是整体能保证90分)，重力抗翻转力矩如下

In [4]:
# 常量定义
g = 9.8  # 重力加速度 (m/s^2)
L = 1.03  # 重力的恢复力臂 (m)
M = 1300  # 平台总质量kg
MG = M * g * L
print("重力抗翻转力矩 MG =", MG, "Nm")

重力抗翻转力矩 MG = 13122.200000000003 Nm


# 防沉板配置

## 1-8全填0

### 输入抗剪强度

In [5]:
Shear_strength = 3.3

In [6]:
if Shear_strength == 0.8:
    C = 800  # 沉积物抗剪强度 (Pa)，将kPa转换为Pa
    gamma1 = 3800  # 土的有效重度 (N/m^3)，将kN/m^3转换为N/m^3
elif Shear_strength == 3.3:
    C = 3300  # 沉积物抗剪强度 (Pa)，将kPa转换为Pa
    gamma1 = 5500  # 土的有效重度 (N/m^3)，将kN/m^3转换为N/m^3

In [7]:
H = 0.06  # 平台底面被覆土平均厚度 (m)
# 固定防沉板参数
B1_fixed = 0.56  # 固定防沉板接触面宽度 (m)
L1_fixed = 0.56  # 固定防沉板接触面长度 (m)
S_fixed = 0.3  # 固定防沉板接触面积 (m^2)
# 中间仪器固定板参数
B1_instrument = 1.06  # 中间仪器固定板接触面宽度 (m)
L1_instrument = 1.06  # 中间仪器固定板接触面长度 (m)
S_instrument = 1.0  # 中间仪器固定板接触面积 (m^2)
# 活动防沉板参数
B1_movable = 0.56  # 活动防沉板接触面宽度 (m)
L1_movable = 0.56  # 活动防沉板接触面长度 (m)
# 初始化活动防沉板配置块数（通过改变数量计算）
num_blocks_movable = 0
def calculate_Nc(B1, L1):
    return 5.0 * (1 + 0.2 * H / B1) * (1 + 0.2 * B1 / L1)
def calculate_Q(B1, L1, S):
    Nc = calculate_Nc(B1, L1)
    return (C * Nc + gamma1 * H) * S
def calculate_total_capacity(num_blocks_movable):
    Q_fixed = calculate_Q(B1_fixed, L1_fixed, S_fixed) * 4
    Q_instrument = calculate_Q(B1_instrument, L1_instrument, S_instrument)
    Q_movable = calculate_Q(B1_movable, L1_movable, S_fixed )* num_blocks_movable
    total_capacity = Q_fixed + Q_instrument + Q_movable
    return total_capacity


# 打印结果
print("总承载力:", calculate_total_capacity(num_blocks_movable), "N")


总承载力: 45019.293800539075 N


# 缆绳选择


In [8]:
### 最小起吊拉力
Fmin = M * g
print("最小起吊拉力(N)：", Fmin)

最小起吊拉力(N)： 12740.000000000002


In [9]:
import math

# 常量定义
C_sediment = Shear_strength * 1000  # 沉积物抗剪强度 (Pa)，将kPa转换为Pa
D_depth = 0.06  # 结构物在底质中的浸没深度 (m)
S_fixed = 1.2  # 固定防沉板接触面积 (m^2)
t_fixed = 0.01  # 防沉板的厚度 (m)
# 计算沉积物对平台的总吸附力
def calculate_total_adhesion_force(B, L, S, t):
    De = D_depth * math.exp(math.sqrt(D_depth))
    SR = S
    Sp = S
    adhesion_force = 5 * S * C_sediment * (1.0 + 0.2 * De / B) * (1.0 + 0.2 * B / L) * math.sin(math.pi * SR / (2 * Sp))

    return adhesion_force
# 固定防沉板参数
B_fixed = 0.56  # 固定防沉板接触面宽度 (m)
L_fixed = 0.56  # 固定防沉板接触面长度 (m)
adhesion_force_fixed = calculate_total_adhesion_force(B_fixed, L_fixed, S_fixed, t_fixed)
# 中间仪器固定板参数
B1_instrument = 1.06  # 中间仪器固定板接触面宽度 (m)
L1_instrument = 1.06  # 中间仪器固定板接触面长度 (m)
S_instrument = 1.0  # 中间仪器固定板接触面积 (m^2)
adhesion_force_instrument = calculate_total_adhesion_force(B1_instrument, L1_instrument, S_instrument, t_fixed)
# 活动防沉板参数
B1_movable = 0.56  # 活动防沉板接触面宽度 (m)
L1_movable = 0.56  # 活动防沉板接触面长度 (m)
# 初始化活动防沉板配置块数
num_blocks_movable = 0
adhesion_force_movable = calculate_total_adhesion_force(B1_movable, L1_movable, S_fixed, t_fixed) * num_blocks_movable
total_adhesion_force = adhesion_force_fixed + adhesion_force_instrument + adhesion_force_movable
print("固定防沉板吸附力（N）:", adhesion_force_fixed)
print("中间仪器固定板吸附力（N）:", adhesion_force_instrument)
print("活动防沉板吸附力（N）:", adhesion_force_movable)
print("总吸附力（N）:", total_adhesion_force)

固定防沉板吸附力（N）: 24410.458574786757
中间仪器固定板吸附力（N）: 20086.365410283473
活动防沉板吸附力（N）: 0.0
总吸附力（N）: 44496.82398507023


In [10]:
Fmax = Fmin + total_adhesion_force
print("回收最小拉力(N)：", Fmax)

回收最小拉力(N)： 57236.82398507023


## 最后别忘记选缆绳哦
## 缆绳选择按稍大于回收最小拉力的就行